<a href="https://colab.research.google.com/github/hansuho113/SwUniv_CancerAnalysis/blob/main/Colab_Notebook/HeeSeon/Randomforest_HeeSeon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [91]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn import metrics
from sklearn.metrics import accuracy_score

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [6]:
drug = pd.read_csv("/content/drive/MyDrive/2021캡스톤/drug_gene_preprocessed.csv")
drug

,index,bcr_patient_barcode,measure_of_response,regimen_indication,therapy_types,drug_name_,therapy_response,?|100130426,?|100133144,?|100134869,?|10357,?|10431,?|136542,?|155060,?|26823,?|280660,?|317712,?|340602,?|388795,?|390284,?|391343,?|391714,?|404770,?|441362,?|442388,?|553137,?|57714,?|645851,?|652919,?|653553,?|728045,?|728603,?|728788,?|729884,?|8225,?|90288,A1BG|1,A1CF|29974,A2BP1|54715,A2LD1|87769,...,ZPBP2|124626,ZPLD1|131368,ZRANB1|54764,ZRANB2|9406,ZRANB3|84083,ZRSR2|8233,ZSCAN10|84891,ZSCAN1|284312,ZSCAN12|9753,ZSCAN12P1|221584,ZSCAN16|80345,ZSCAN18|65982,ZSCAN20|7579,ZSCAN21|7589,ZSCAN22|342945,ZSCAN23|222696,ZSCAN2|54993,ZSCAN29|146050,ZSCAN4|201516,ZSCAN5A|79149,ZSCAN5B|342933,ZSWIM1|90204,ZSWIM2|151112,ZSWIM3|140831,ZSWIM4|65249,ZSWIM5|57643,ZSWIM6|57688,ZSWIM7|125150,ZUFSP|221302,ZW10|9183,ZWILCH|55055,ZWINT|11130,ZXDA|7789,ZXDB|158586,ZXDC|79364,ZYG11A|440590,ZYG11B|79699,ZYX|7791,ZZEF1|23140,ZZZ3|26009
0,TCGA-A6-6648-01A-11R-1774-07,TCGA-A6-6648,unknown,"OTHER, SPECIFY IN NOTES",unknown,oxaliplatin,unknown,0.0,14.170300,12.259200,324.308000,713.850000,0.0,152.732000,0.5083,0.0,0.0,0.0,3.049600,2.033000,0.0,0.0,0.0,0.0,0.0,0.0,732.147000,12.452400,0.0000,31.512100,0.0,0.0,2.0330,0.508300,506.480000,0.508300,6.792900,191.614000,6.861500,101.741000,...,0.0000,0.2541,737.992000,1298.830000,116.900000,160.864000,0.000000,0.254100,224.905000,33.545100,178.145000,461.245000,85.641700,142.473000,158.831000,4.32020,134.943000,722.745000,0.0000,134.435000,1.77890,518.678000,0.0,309.022000,240.661000,703.939000,488.437000,110.06600,233.799000,892.503000,940.45500,1146.630000,66.836100,558.577000,988.310000,1.016500,1029.220000,2791.870000,2283.100000,1205.5900
1,TCGA-A6-6648-01A-11R-1774-07,TCGA-A6-6648,unknown,"OTHER, SPECIFY IN NOTES",unknown,folinic,unknown,0.0,14.170300,12.259200,324.308000,713.850000,0.0,152.732000,0.5083,0.0,0.0,0.0,3.049600,2.033000,0.0,0.0,0.0,0.0,0.0,0.0,732.147000,12.452400,0.0000,31.512100,0.0,0.0,2.0330,0.508300,506.480000,0.508300,6.792900,191.614000,6.861500,101.741000,...,0.0000,0.2541,737.992000,1298.830000,116.900000,160.864000,0.000000,0.254100,224.905000,33.545100,178.145000,461.245000,85.641700,142.473000,158.831000,4.32020,134.943000,722.745000,0.0000,134.435000,1.77890,518.678000,0.0,309.022000,240.661000,703.939000,488.437000,110.06600,233.799000,892.503000,940.45500,1146.630000,66.836100,558.577000,988.310000,1.016500,1029.220000,2791.870000,2283.100000,1205.5900
2,TCGA-A6-6648-01A-11R-1774-07,TCGA-A6-6648,unknown,"OTHER, SPECIFY IN NOTES",unknown,fluorouracil,unknown,0.0,14.170300,12.259200,324.308000,713.850000,0.0,152.732000,0.5083,0.0,0.0,0.0,3.049600,2.033000,0.0,0.0,0.0,0.0,0.0,0.0,732.147000,12.452400,0.0000,31.512100,0.0,0.0,2.0330,0.508300,506.480000,0.508300,6.792900,191.614000,6.861500,101.741000,...,0.0000,0.2541,737.992000,1298.830000,116.900000,160.864000,0.000000,0.254100,224.905000,33.545100,178.145000,461.245000,85.641700,142.473000,158.831000,4.32020,134.943000,722.745000,0.0000,134.435000,1.77890,518.678000,0.0,309.022000,240.661000,703.939000,488.437000,110.06600,233.799000,892.503000,940.45500,1146.630000,66.836100,558.577000,988.310000,1.016500,1029.220000,2791.870000,2283.100000,1205.5900
3,TCGA-AZ-4615-01A-01R-1410-07,TCGA-AZ-4615,unknown,ADJUVANT,Chemotherapy,capecitabine,unknown,0.0,25.104200,23.888800,258.457000,1157.460000,0.0,59.828100,0.4711,0.0,0.0,0.0,0.000000,8.950700,0.0,0.0,0.0,0.0,0.0,0.0,975.150000,1.884300,0.0000,324.579000,0.0,0.0,0.4711,0.000000,801.319000,0.471100,24.162100,11.306100,0.000000,165.036000,...,0.4711,0.0000,518.196000,836.085000,30.620700,104.110000,0.000000,0.000000,60.299100,63.530800,97.043900,98.457200,45.695400,122.030000,84.795700,1.41330,141.326000,452.715000,3.7687,82.440200,1.88430,206.336000,0.0,71.605200,203.981000,190.790000,268.048000,114.66300,199.741000,762.690000,602.88300,1385.000000,57.001500,377.812000,835.237000,0.471100,598.280000,5186.670000,1034.510000,522.4360
4,TCGA-G4-6297-01A-11R-1723-07,TCGA-G

In [36]:
drug_r = drug.drop(['?|100130426', '?|100133144', '?|100134869', '?|10357', '?|10431', '?|136542', '?|155060', '?|26823', '?|280660', '?|317712', '?|340602', '?|388795',
                       '?|390284', '?|391343', '?|391714', '?|404770', '?|441362', '?|442388', '?|553137', '?|57714', '?|645851', '?|652919', '?|653553', 
                       '?|728045', '?|728603', '?|728788', '?|729884', '?|8225', '?|90288'], axis=1)
drug_r

,index,bcr_patient_barcode,measure_of_response,regimen_indication,therapy_types,drug_name_,therapy_response,A1BG|1,A1CF|29974,A2BP1|54715,A2LD1|87769,A2M|2,A2ML1|144568,A4GALT|53947,A4GNT|51146,AAA1|404744,AAAS|8086,AACS|65985,AACSL|729522,AADAC|13,AADACL2|344752,AADACL3|126767,AADACL4|343066,AADAT|51166,AAGAB|79719,AAK1|22848,AAMP|14,AANAT|15,AARS|16,AARS2|57505,AARSD1|80755,AASDH|132949,AASDHPPT|60496,AASS|10157,AATF|26574,AATK|9625,ABAT|18,ABCA10|10349,ABCA1|19,ABCA11P|79963,...,ZPBP2|124626,ZPLD1|131368,ZRANB1|54764,ZRANB2|9406,ZRANB3|84083,ZRSR2|8233,ZSCAN10|84891,ZSCAN1|284312,ZSCAN12|9753,ZSCAN12P1|221584,ZSCAN16|80345,ZSCAN18|65982,ZSCAN20|7579,ZSCAN21|7589,ZSCAN22|342945,ZSCAN23|222696,ZSCAN2|54993,ZSCAN29|146050,ZSCAN4|201516,ZSCAN5A|79149,ZSCAN5B|342933,ZSWIM1|90204,ZSWIM2|151112,ZSWIM3|140831,ZSWIM4|65249,ZSWIM5|57643,ZSWIM6|57688,ZSWIM7|125150,ZUFSP|221302,ZW10|9183,ZWILCH|55055,ZWINT|11130,ZXDA|7789,ZXDB|158586,ZXDC|79364,ZYG11A|440590,ZYG11B|79699,ZYX|7791,ZZEF1|23140,ZZZ3|26009
0,TCGA-A6-6648-01A-11R-1774-07,TCGA-A6-6648,unknown,"OTHER, SPECIFY IN NOTES",unknown,oxaliplatin,unknown,6.792900,191.614000,6.861500,101.741000,4753.24000,0.254100,48.030500,0.0000,0.2541,748.920000,951.207000,0.0000,3.55780,0.2541,0.0,0.0,215.248000,1728.080000,995.680000,2034.31000,0.254100,2742.820000,702.900000,718.170000,212.707000,1094.280000,79.034300,1883.610000,48.030500,855.146000,7.623900,616.76800,84.607400,...,0.0000,0.2541,737.992000,1298.830000,116.900000,160.864000,0.000000,0.254100,224.905000,33.545100,178.145000,461.245000,85.641700,142.473000,158.831000,4.32020,134.943000,722.745000,0.0000,134.435000,1.77890,518.678000,0.0,309.022000,240.661000,703.939000,488.437000,110.06600,233.799000,892.503000,940.45500,1146.630000,66.836100,558.577000,988.310000,1.016500,1029.220000,2791.870000,2283.100000,1205.5900
1,TCGA-A6-6648-01A-11R-1774-07,TCGA-A6-6648,unknown,"OTHER, SPECIFY IN NOTES",unknown,folinic,unknown,6.792900,191.614000,6.861500,101.741000,4753.24000,0.254100,48.030500,0.0000,0.2541,748.920000,951.207000,0.0000,3.55780,0.2541,0.0,0.0,215.248000,1728.080000,995.680000,2034.31000,0.254100,2742.820000,702.900000,718.170000,212.707000,1094.280000,79.034300,1883.610000,48.030500,855.146000,7.623900,616.76800,84.607400,...,0.0000,0.2541,737.992000,1298.830000,116.900000,160.864000,0.000000,0.254100,224.905000,33.545100,178.145000,461.245000,85.641700,142.473000,158.831000,4.32020,134.943000,722.745000,0.0000,134.435000,1.77890,518.678000,0.0,309.022000,240.661000,703.939000,488.437000,110.06600,233.799000,892.503000,940.45500,1146.630000,66.836100,558.577000,988.310000,1.016500,1029.220000,2791.870000,2283.100000,1205.5900
2,TCGA-A6-6648-01A-11R-1774-07,TCGA-A6-6648,unknown,"OTHER, SPECIFY IN NOTES",unknown,fluorouracil,unknown,6.792900,191.614000,6.861500,101.741000,4753.24000,0.254100,48.030500,0.0000,0.2541,748.920000,951.207000,0.0000,3.55780,0.2541,0.0,0.0,215.248000,1728.080000,995.680000,2034.31000,0.254100,2742.820000,702.900000,718.170000,212.707000,1094.280000,79.034300,1883.610000,48.030500,855.146000,7.623900,616.76800,84.607400,...,0.0000,0.2541,737.992000,1298.830000,116.900000,160.864000,0.000000,0.254100,224.905000,33.545100,178.145000,461.245000,85.641700,142.473000,158.831000,4.32020,134.943000,722.745000,0.0000,134.435000,1.77890,518.678000,0.0,309.022000,240.661000,703.939000,488.437000,110.06600,233.799000,892.503000,940.45500,1146.630000,66.836100,558.577000,988.310000,1.016500,1029.220000,2791.870000,2283.100000,1205.5900
3,TCGA-AZ-4615-01A-01R-1410-07,TCGA-AZ-4615,unknown,ADJUVANT,Chemotherapy,capecitabine,unknown,24.162100,11.306100,0.000000,165.036000,3868.09000,0.000000,169.120000,1.4133,5.1820,815.452000,819.220000,0.0000,13.19040,0.0000,0.0,0.0,238.841000,1538.570000,421.152000,3526.56000,0.000000,4388.180000,430.569000,371.217000,130.020000,668.439000,32.033900,1623.370000,21.198900,516.783000,3.297600,413.14300,109.707000,...,0.4711,0.0000,518.196000,836.085000,30.620700,104.110000,0.0000

In [37]:
drug_r = drug_r.drop(['bcr_patient_barcode'], axis=1)
drug_r.head()

,index,measure_of_response,regimen_indication,therapy_types,drug_name_,therapy_response,A1BG|1,A1CF|29974,A2BP1|54715,A2LD1|87769,A2M|2,A2ML1|144568,A4GALT|53947,A4GNT|51146,AAA1|404744,AAAS|8086,AACS|65985,AACSL|729522,AADAC|13,AADACL2|344752,AADACL3|126767,AADACL4|343066,AADAT|51166,AAGAB|79719,AAK1|22848,AAMP|14,AANAT|15,AARS|16,AARS2|57505,AARSD1|80755,AASDH|132949,AASDHPPT|60496,AASS|10157,AATF|26574,AATK|9625,ABAT|18,ABCA10|10349,ABCA1|19,ABCA11P|79963,ABCA12|26154,...,ZPBP2|124626,ZPLD1|131368,ZRANB1|54764,ZRANB2|9406,ZRANB3|84083,ZRSR2|8233,ZSCAN10|84891,ZSCAN1|284312,ZSCAN12|9753,ZSCAN12P1|221584,ZSCAN16|80345,ZSCAN18|65982,ZSCAN20|7579,ZSCAN21|7589,ZSCAN22|342945,ZSCAN23|222696,ZSCAN2|54993,ZSCAN29|146050,ZSCAN4|201516,ZSCAN5A|79149,ZSCAN5B|342933,ZSWIM1|90204,ZSWIM2|151112,ZSWIM3|140831,ZSWIM4|65249,ZSWIM5|57643,ZSWIM6|57688,ZSWIM7|125150,ZUFSP|221302,ZW10|9183,ZWILCH|55055,ZWINT|11130,ZXDA|7789,ZXDB|158586,ZXDC|79364,ZYG11A|440590,ZYG11B|79699,ZYX|7791,ZZEF1|23140,ZZZ3|26009
0,TCGA-A6-6648-01A-11R-1774-07,unknown,"OTHER, SPECIFY IN NOTES",unknown,oxaliplatin,unknown,6.7929,191.6140,6.8615,101.741,4753.24,0.2541,48.0305,0.0000,0.2541,748.920,951.207,0.0,3.5578,0.2541,0.0,0.0,215.248,1728.08,995.680,2034.31,0.2541,2742.82,702.900,718.170,212.707,1094.280,79.0343,1883.61,48.0305,855.146,7.6239,616.768,84.6074,3.3037,...,0.0000,0.2541,737.992,1298.830,116.9000,160.864,0.0,0.2541,224.9050,33.5451,178.1450,461.2450,85.6417,142.473,158.8310,4.3202,134.943,722.745,0.0000,134.4350,1.7789,518.678,0.0,309.0220,240.661,703.939,488.437,110.066,233.799,892.503,940.455,1146.63,66.8361,558.577,988.310,1.0165,1029.220,2791.87,2283.10,1205.590
1,TCGA-A6-6648-01A-11R-1774-07,unknown,"OTHER, SPECIFY IN NOTES",unknown,folinic,unknown,6.7929,191.6140,6.8615,101.741,4753.24,0.2541,48.0305,0.0000,0.2541,748.920,951.207,0.0,3.5578,0.2541,0.0,0.0,215.248,1728.08,995.680,2034.31,0.2541,2742.82,702.900,718.170,212.707,1094.280,79.0343,1883.61,48.0305,855.146,7.6239,616.768,84.6074,3.3037,...,0.0000,0.2541,737.992,1298.830,116.9000,160.864,0.0,0.2541,224.9050,33.5451,178.1450,461.2450,85.6417,142.473,158.8310,4.3202,134.943,722.745,0.0000,134.4350,1.7789,518.678,0.0,309.0220,240.661,703.939,488.437,110.066,233.799,892.503,940.455,1146.63,66.8361,558.577,988.310,1.0165,1029.220,2791.87,2283.10,1205.590
2,TCGA-A6-6648-01A-11R-1774-07,unknown,"OTHER, SPECIFY IN NOTES",unknown,fluorouracil,unknown,6.7929,191.6140,6.8615,101.741,4753.24,0.2541,48.0305,0.0000,0.2541,748.920,951.207,0.0,3.5578,0.2541,0.0,0.0,215.248,1728.08,995.680,2034.31,0.2541,2742.82,702.900,718.170,212.707,1094.280,79.0343,1883.61,48.0305,855.146,7.6239,616.768,84.6074,3.3037,...,0.0000,0.2541,737.992,1298.830,116.9000,160.864,0.0,0.2541,224.9050,33.5451,178.1450,461.2450,85.6417,142.473,158.8310,4.3202,134.943,722.745,0.0000,134.4350,1.7789,518.678,0.0,309.0220,240.661,703.939,488.437,110.066,233.799,892.503,940.455,1146.63,66.8361,558.577,988.310,1.0165,1029.220,2791.87,2283.10,1205.590
3,TCGA-AZ-4615-01A-01R-1410-07,unknown,ADJUVANT,Chemotherapy,capecitabine,unknown,24.1621,11.3061,0.0000,165.036,3868.09,0.0000,169.1200,1.4133,5.1820,815.452,819.220,0.0,13.1904,0.0000,0.0,0.0,238.841,1538.57,421.152,3526.56,0.0000,4388.18,430.569,371.217,130.020,668.439,32.0339,1623.37,21.1989,516.783,3.2976,413.143,109.7070,8.4796,...,0.4711,0.0000,518.196,836.085,30.6207,104.110,0.0,0.0000,60.2991,63.5308,97.0439,98.4572,45.6954,122.030,84.7957,1.4133,141.326,452.715,3.7687,82.4402,1.8843,206.336,0.0,71.6052,203.981,190.790,268.048,114.663,199.741,762.690,602.883,1385.00,57.0015,377.812,835.237,0.4711,598.280,5186.67,1034.51,522.436
4,TCGA-G4-6297-01A-11R-1723-07,unknown,ADJUVANT,Chemotherapy,oxaliplatin,unknown,27.3250,81.9751,33.5570,242.047,8580.50,11.0259,359.5400,0.9588,0.4794,567.114,744.008,0.0,20.6136,0.0000,0.0,0.0,113.615,1181.69,1026.850,2261.27,0.9588,2924.26,787.143,616.970,177.373,546.500,165.3880,1553.21,383.5090,691.275,18.2167,778.989,75.7574,0.4794,...,0.4794,1.4382,573.82

In [38]:
drug_r.rename(columns = {"index" : "barcode"},inplace=True)
drug_r.head()

,barcode,measure_of_response,regimen_indication,therapy_types,drug_name_,therapy_response,A1BG|1,A1CF|29974,A2BP1|54715,A2LD1|87769,A2M|2,A2ML1|144568,A4GALT|53947,A4GNT|51146,AAA1|404744,AAAS|8086,AACS|65985,AACSL|729522,AADAC|13,AADACL2|344752,AADACL3|126767,AADACL4|343066,AADAT|51166,AAGAB|79719,AAK1|22848,AAMP|14,AANAT|15,AARS|16,AARS2|57505,AARSD1|80755,AASDH|132949,AASDHPPT|60496,AASS|10157,AATF|26574,AATK|9625,ABAT|18,ABCA10|10349,ABCA1|19,ABCA11P|79963,ABCA12|26154,...,ZPBP2|124626,ZPLD1|131368,ZRANB1|54764,ZRANB2|9406,ZRANB3|84083,ZRSR2|8233,ZSCAN10|84891,ZSCAN1|284312,ZSCAN12|9753,ZSCAN12P1|221584,ZSCAN16|80345,ZSCAN18|65982,ZSCAN20|7579,ZSCAN21|7589,ZSCAN22|342945,ZSCAN23|222696,ZSCAN2|54993,ZSCAN29|146050,ZSCAN4|201516,ZSCAN5A|79149,ZSCAN5B|342933,ZSWIM1|90204,ZSWIM2|151112,ZSWIM3|140831,ZSWIM4|65249,ZSWIM5|57643,ZSWIM6|57688,ZSWIM7|125150,ZUFSP|221302,ZW10|9183,ZWILCH|55055,ZWINT|11130,ZXDA|7789,ZXDB|158586,ZXDC|79364,ZYG11A|440590,ZYG11B|79699,ZYX|7791,ZZEF1|23140,ZZZ3|26009
0,TCGA-A6-6648-01A-11R-1774-07,unknown,"OTHER, SPECIFY IN NOTES",unknown,oxaliplatin,unknown,6.7929,191.6140,6.8615,101.741,4753.24,0.2541,48.0305,0.0000,0.2541,748.920,951.207,0.0,3.5578,0.2541,0.0,0.0,215.248,1728.08,995.680,2034.31,0.2541,2742.82,702.900,718.170,212.707,1094.280,79.0343,1883.61,48.0305,855.146,7.6239,616.768,84.6074,3.3037,...,0.0000,0.2541,737.992,1298.830,116.9000,160.864,0.0,0.2541,224.9050,33.5451,178.1450,461.2450,85.6417,142.473,158.8310,4.3202,134.943,722.745,0.0000,134.4350,1.7789,518.678,0.0,309.0220,240.661,703.939,488.437,110.066,233.799,892.503,940.455,1146.63,66.8361,558.577,988.310,1.0165,1029.220,2791.87,2283.10,1205.590
1,TCGA-A6-6648-01A-11R-1774-07,unknown,"OTHER, SPECIFY IN NOTES",unknown,folinic,unknown,6.7929,191.6140,6.8615,101.741,4753.24,0.2541,48.0305,0.0000,0.2541,748.920,951.207,0.0,3.5578,0.2541,0.0,0.0,215.248,1728.08,995.680,2034.31,0.2541,2742.82,702.900,718.170,212.707,1094.280,79.0343,1883.61,48.0305,855.146,7.6239,616.768,84.6074,3.3037,...,0.0000,0.2541,737.992,1298.830,116.9000,160.864,0.0,0.2541,224.9050,33.5451,178.1450,461.2450,85.6417,142.473,158.8310,4.3202,134.943,722.745,0.0000,134.4350,1.7789,518.678,0.0,309.0220,240.661,703.939,488.437,110.066,233.799,892.503,940.455,1146.63,66.8361,558.577,988.310,1.0165,1029.220,2791.87,2283.10,1205.590
2,TCGA-A6-6648-01A-11R-1774-07,unknown,"OTHER, SPECIFY IN NOTES",unknown,fluorouracil,unknown,6.7929,191.6140,6.8615,101.741,4753.24,0.2541,48.0305,0.0000,0.2541,748.920,951.207,0.0,3.5578,0.2541,0.0,0.0,215.248,1728.08,995.680,2034.31,0.2541,2742.82,702.900,718.170,212.707,1094.280,79.0343,1883.61,48.0305,855.146,7.6239,616.768,84.6074,3.3037,...,0.0000,0.2541,737.992,1298.830,116.9000,160.864,0.0,0.2541,224.9050,33.5451,178.1450,461.2450,85.6417,142.473,158.8310,4.3202,134.943,722.745,0.0000,134.4350,1.7789,518.678,0.0,309.0220,240.661,703.939,488.437,110.066,233.799,892.503,940.455,1146.63,66.8361,558.577,988.310,1.0165,1029.220,2791.87,2283.10,1205.590
3,TCGA-AZ-4615-01A-01R-1410-07,unknown,ADJUVANT,Chemotherapy,capecitabine,unknown,24.1621,11.3061,0.0000,165.036,3868.09,0.0000,169.1200,1.4133,5.1820,815.452,819.220,0.0,13.1904,0.0000,0.0,0.0,238.841,1538.57,421.152,3526.56,0.0000,4388.18,430.569,371.217,130.020,668.439,32.0339,1623.37,21.1989,516.783,3.2976,413.143,109.7070,8.4796,...,0.4711,0.0000,518.196,836.085,30.6207,104.110,0.0,0.0000,60.2991,63.5308,97.0439,98.4572,45.6954,122.030,84.7957,1.4133,141.326,452.715,3.7687,82.4402,1.8843,206.336,0.0,71.6052,203.981,190.790,268.048,114.663,199.741,762.690,602.883,1385.00,57.0015,377.812,835.237,0.4711,598.280,5186.67,1034.51,522.436
4,TCGA-G4-6297-01A-11R-1723-07,unknown,ADJUVANT,Chemotherapy,oxaliplatin,unknown,27.3250,81.9751,33.5570,242.047,8580.50,11.0259,359.5400,0.9588,0.4794,567.114,744.008,0.0,20.6136,0.0000,0.0,0.0,113.615,1181.69,1026.850,2261.27,0.9588,2924.26,787.143,616.970,177.373,546.500,165.3880,1553.21,383.5090,691.275,18.2167,778.989,75.7574,0.4794,...,0.4794,1.4382,573.

In [71]:
encoder = LabelEncoder()
encoder.fit(drug_r.measure_of_response)

labels = encoder.transform(drug_r.measure_of_response)
labels, encoder.classes_

(array([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 1, 1, 2, 2, 2, 2, 2, 0, 0, 0, 4, 4, 4, 4, 4, 4, 4,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4,
        1, 1, 1, 1, 1, 1, 4, 0, 0, 0, 4, 4, 4, 4, 4, 4, 4, 0, 0, 0, 0, 4,
        4, 4, 4, 1, 1, 1, 1, 0, 0, 4, 0, 0, 0, 0, 0, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 1, 2, 2, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 4,
        4, 4, 4, 4, 4, 4, 1, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 1, 1, 1, 3, 4,
        4, 4, 0, 0, 0, 0, 4, 4, 4, 0, 4, 4, 4, 4, 2, 2, 2, 4, 4, 1, 1, 1,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 4, 4, 4, 4,
        4, 2, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4]), array([0, 1, 2, 3, 4]))

In [72]:
drug_r['measure_of_response'] = list(labels)

In [73]:
drug_r

,barcode,measure_of_response,regimen_indication,therapy_types,drug_name_,therapy_response,A1BG|1,A1CF|29974,A2BP1|54715,A2LD1|87769,A2M|2,A2ML1|144568,A4GALT|53947,A4GNT|51146,AAA1|404744,AAAS|8086,AACS|65985,AACSL|729522,AADAC|13,AADACL2|344752,AADACL3|126767,AADACL4|343066,AADAT|51166,AAGAB|79719,AAK1|22848,AAMP|14,AANAT|15,AARS|16,AARS2|57505,AARSD1|80755,AASDH|132949,AASDHPPT|60496,AASS|10157,AATF|26574,AATK|9625,ABAT|18,ABCA10|10349,ABCA1|19,ABCA11P|79963,ABCA12|26154,...,ZPBP2|124626,ZPLD1|131368,ZRANB1|54764,ZRANB2|9406,ZRANB3|84083,ZRSR2|8233,ZSCAN10|84891,ZSCAN1|284312,ZSCAN12|9753,ZSCAN12P1|221584,ZSCAN16|80345,ZSCAN18|65982,ZSCAN20|7579,ZSCAN21|7589,ZSCAN22|342945,ZSCAN23|222696,ZSCAN2|54993,ZSCAN29|146050,ZSCAN4|201516,ZSCAN5A|79149,ZSCAN5B|342933,ZSWIM1|90204,ZSWIM2|151112,ZSWIM3|140831,ZSWIM4|65249,ZSWIM5|57643,ZSWIM6|57688,ZSWIM7|125150,ZUFSP|221302,ZW10|9183,ZWILCH|55055,ZWINT|11130,ZXDA|7789,ZXDB|158586,ZXDC|79364,ZYG11A|440590,ZYG11B|79699,ZYX|7791,ZZEF1|23140,ZZZ3|26009
0,TCGA-A6-6648-01A-11R-1774-07,4,"OTHER, SPECIFY IN NOTES",unknown,oxaliplatin,unknown,6.792900,191.614000,6.861500,101.741000,4753.24000,0.254100,48.030500,0.0000,0.2541,748.920000,951.207000,0.0000,3.55780,0.2541,0.0,0.0,215.248000,1728.080000,995.680000,2034.31000,0.254100,2742.820000,702.900000,718.170000,212.707000,1094.280000,79.034300,1883.610000,48.030500,855.146000,7.623900,616.76800,84.607400,3.303700,...,0.0000,0.2541,737.992000,1298.830000,116.900000,160.864000,0.000000,0.254100,224.905000,33.545100,178.145000,461.245000,85.641700,142.473000,158.831000,4.32020,134.943000,722.745000,0.0000,134.435000,1.77890,518.678000,0.0,309.022000,240.661000,703.939000,488.437000,110.06600,233.799000,892.503000,940.45500,1146.630000,66.836100,558.577000,988.310000,1.016500,1029.220000,2791.870000,2283.100000,1205.5900
1,TCGA-A6-6648-01A-11R-1774-07,4,"OTHER, SPECIFY IN NOTES",unknown,folinic,unknown,6.792900,191.614000,6.861500,101.741000,4753.24000,0.254100,48.030500,0.0000,0.2541,748.920000,951.207000,0.0000,3.55780,0.2541,0.0,0.0,215.248000,1728.080000,995.680000,2034.31000,0.254100,2742.820000,702.900000,718.170000,212.707000,1094.280000,79.034300,1883.610000,48.030500,855.146000,7.623900,616.76800,84.607400,3.303700,...,0.0000,0.2541,737.992000,1298.830000,116.900000,160.864000,0.000000,0.254100,224.905000,33.545100,178.145000,461.245000,85.641700,142.473000,158.831000,4.32020,134.943000,722.745000,0.0000,134.435000,1.77890,518.678000,0.0,309.022000,240.661000,703.939000,488.437000,110.06600,233.799000,892.503000,940.45500,1146.630000,66.836100,558.577000,988.310000,1.016500,1029.220000,2791.870000,2283.100000,1205.5900
2,TCGA-A6-6648-01A-11R-1774-07,4,"OTHER, SPECIFY IN NOTES",unknown,fluorouracil,unknown,6.792900,191.614000,6.861500,101.741000,4753.24000,0.254100,48.030500,0.0000,0.2541,748.920000,951.207000,0.0000,3.55780,0.2541,0.0,0.0,215.248000,1728.080000,995.680000,2034.31000,0.254100,2742.820000,702.900000,718.170000,212.707000,1094.280000,79.034300,1883.610000,48.030500,855.146000,7.623900,616.76800,84.607400,3.303700,...,0.0000,0.2541,737.992000,1298.830000,116.900000,160.864000,0.000000,0.254100,224.905000,33.545100,178.145000,461.245000,85.641700,142.473000,158.831000,4.32020,134.943000,722.745000,0.0000,134.435000,1.77890,518.678000,0.0,309.022000,240.661000,703.939000,488.437000,110.06600,233.799000,892.503000,940.45500,1146.630000,66.836100,558.577000,988.310000,1.016500,1029.220000,2791.870000,2283.100000,1205.5900
3,TCGA-AZ-4615-01A-01R-1410-07,4,ADJUVANT,Chemotherapy,capecitabine,unknown,24.162100,11.306100,0.000000,165.036000,3868.09000,0.000000,169.120000,1.4133,5.1820,815.452000,819.220000,0.0000,13.19040,0.0000,0.0,0.0,238.841000,1538.570000,421.152000,3526.56000,0.000000,4388.180000,430.569000,371.217000,130.020000,668.439000,32.033900,1623.370000,21.198900,516.783000,3.297600,413.14300,109.707000,8.479600,...,0.4711,0.0000,518.196000,836.085000,30.620700,104.110000,0.000000,0.000000,60.299100,63.530800,97.043900,98.

In [74]:
dn = pd.get_dummies(drug_r['drug_name_'], prefix="drugname")
ri = pd.get_dummies(drug_r['regimen_indication'], prefix="indication")
tt = pd.get_dummies(drug_r['therapy_types'], prefix="therapy_types")

In [76]:
drugs = pd.concat([drug_r,dn,ri,tt], axis=1)
drugs.shape

(245, 20527)

In [77]:
X = drug_r[drug_r.columns.difference(["barcode", 'measure_of_response', "regimen_indication", "therapy_types", "drug_name_", "therapy_response"])]
y = drug_r[['measure_of_response']]

In [93]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

rnd_clf = RandomForestClassifier(n_estimators=100, random_state=42)

rnd_clf.fit(X_train, y_train)

print("Accuracy on training set: {:.3f}".format(rnd_clf.score(X_train, y_train)))
print("Accuracy on test set: {:.3f}".format(rnd_clf.score(X_test, y_test)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


Accuracy on training set: 0.924
Accuracy on test set: 0.838


In [98]:
y_pred = rnd_clf.predict(X_test)
print("RandomForesClassifier Predict Accuracy : ", metrics.accuracy_score(y_test, y_pred))

RandomForesClassifier Predict Accuracy :  0.8378378378378378


In [99]:
rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, random_state=42)
rnd_clf.fit(X_train, y_train)

y_pred_rf = rnd_clf.predict(X_test)
print("RandomForesClassifier Predict Accuracy : ", metrics.accuracy_score(y_test, y_pred_rf))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForesClassifier Predict Accuracy :  0.7972972972972973


In [86]:
np.sum(y_pred == y_pred_rf) / len(y_pred)  # almost identical predictions

1.0

bagging

In [83]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(splitter="random", max_leaf_nodes=16, random_state=42),
    n_estimators=500, max_samples=1.0, bootstrap=True, random_state=42)

In [92]:
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)
print("Predict Accuracy", metrics.accuracy_score(y_test, y_pred))

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_bagging.py:645: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Predict Accuracy 0.7972972972972973
